<a href="https://colab.research.google.com/github/josh-ashkinaze/reddit_rship_advice/blob/master/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# New Analysis: GET Data

In [0]:
!pip install praw 
!pip install sklearn 



import praw 
import pandas as pd
from google.colab import files
from IPython.display import display, clear_output 
import warnings



CLIENT_ID = 'LrnElYx-o4jf8g'
SECRET_ID = 'UERC-XdvxAeVWjdPi3vro8CE8o4'
PW = 'elasticmind'
USERNAME = 't3cblaze'
reddit = praw.Reddit(
                     client_id=CLIENT_ID,
                     client_secret= SECRET_ID,
                     password=PW,
                     user_agent='testscript by /u/fakebot3',
                     username=USERNAME
                     )

def config():
  warnings.filterwarnings("ignore", category=DeprecationWarning)
  clear_output()

def generate_posts(subreddit, nposts):
  sample = []
  sub = reddit.subreddit(subreddit)
  for i in range(nposts):
    random_post = sub.random()
    sample.append(random_post)
    print(i)
  return sample
  
def parse_submissions(submissions, comment_limit): 
  submission_count = 0
  all_post_data = []
  added_posts = []
  for post in submissions:
    if not(already_added(added_posts, post.title)):
      post_data = get_post_data(post, comment_limit)
      all_post_data.append(post_data)
      submission_count+=1
      print("POST {} OF MAX: {}".format(submission_count, len(submissions)))
  return all_post_data


# Check if item in list and update list if not
def already_added(added_list, item):
  if item in added_list:
    return True
  else: 
    added_list.append(item)
    return False

# Returns comments of post of level <= comment_limit
def get_post_comments(post, comment_limit):
  post.comments.replace_more(limit=comment_limit)
  comments = [comment.body.strip("[").strip("]") for comment in post.comments.list()]
  return comments

# Returns info about a post 
def get_post_data(post, comment_limit):
  post_info = {
    "title": post.title,
    "flair": post.link_flair_text	,
    "post_id": post.id,
    'self_txt' : post.selftext,
    "score": post.score,
    "upvote_ratio": post.upvote_ratio,
    "num_comments" : post.num_comments,
#     "comments": get_post_comments(post, comment_limit)
  }
  return post_info



def main():
  config()
  sample = generate_posts('relationship_advice', 2)
  post_data = parse_submissions(sample, 1)
  df = pd.DataFrame(post_data)
  df.to_csv("data9.csv")
  files.download("data9.csv")
  

 
  
main()

0
1
POST 1 OF MAX: 2
POST 2 OF MAX: 2


# New Analysis: Analyze

In [3]:
from google.colab import files
d5 = files.upload()


Saving rship.csv to rship.csv


In [0]:

!pip install ekphrasis
!pip install textblob
!pip install contractions

# Utilities 
import re
import string 
from IPython.display import display, clear_output

# NLP Libraries 
import contractions
from ekphrasis.classes.spellcorrect import SpellCorrector
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.decomposition import NMF, LatentDirichletAllocation

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.datasets import fetch_20newsgroups
from textblob import TextBlob

# Data Handling 
import pandas as pd




nltk.download("stopwords")
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:

!pip install ekphrasis
!pip install textblob
!pip install contractions

# Utilities 
import re
import string 
from IPython.display import display, clear_output
import numpy as np

# NLP Libraries 
import contractions
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from textblob import TextBlob

# Data Handling 
import pandas as pd




nltk.download("stopwords")
nltk.download('punkt')


# Print results of topic analysis 
def display_topics(H, W, feature_names, documents, no_top_words, no_top_documents):
  for topic_idx, topic in enumerate(H):
      print ("Topic {}: ".format(topic_idx)  + ",".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))
      top_doc_indices = np.argsort( W[:,topic_idx] )[::-1][0:no_top_documents]
      for idx in top_doc_indices:
          print (documents[idx])
      print()

            
# Build a topic analysis model 
def make_topics(documents, clean_docs):
  
  no_topics = 4 # hypothesized latent topics 
  no_top_words = 7 # number of sample words per topic 
  no_top_documents = 4 # number of docs to print per topic 

  # Fit a TFIDF vectorizer 
  tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, stop_words='english', ngram_range = (1, 1))
  tfidf = tfidf_vectorizer.fit_transform(clean_docs)
  tfidf_feature_names = tfidf_vectorizer.get_feature_names()

  # Run NMF
  nmf_model = NMF(n_components=no_topics, random_state=1, alpha=0.01,l1_ratio=0.5, init='nndsvd').fit(tfidf)
  nmf_W = nmf_model.transform(tfidf)
  print(pd.DataFrame(nmf_W))
  nmf_H = nmf_model.components_

  # Display topics 
  display_topics(nmf_H, nmf_W, tfidf_feature_names, documents, no_top_words, no_top_documents)
  return nmf_H, nmf_W



# Returns a tokenized form of a string 
def tokenize(text):
  custom_sw = {"feel", 'im'}
  stop_words = set(stopwords.words("english")).union(custom_sw)
  nltk_tokenizer = nltk.WordPunctTokenizer().tokenize
  tokens = nltk_tokenizer(text)
  result = " ".join([i for i in tokens if not i in stop_words if len(i)>=1])
  return result

# Cleans a string by applying various pre-processing steps
def clean_text(text):
  new_text = make_lowercase(text)
  new_text = expand_contractions(new_text)
  new_text = remove_punct(new_text)
#   new_text = remove_numbers(new_text)
  return new_text

def correct_spelling(text):
  new_text = str(TextBlob(text).correct())
  return new_text

def make_lowercase(text):
  new_text = text.lower()
  return new_text

def expand_contractions(text):
  new_text = contractions.fix(text)
  return new_text

def remove_punct(text):
  remove_punct_map = dict.fromkeys(map(ord, string.punctuation))
  text.translate(remove_punct_map)
  return text

def remove_numbers(text):
  new_text =re.sub("\d+", "", text)
  return new_text

<class 'str'>


### The very first thing I did was clean the text data 

After this, we can explore the topics of questions. 

In [0]:
# Returns a tokenized form of a string 
def tokenize(text):
  custom_sw = {"feel", 'im'}
  stop_words = set(stopwords.words("english")).union(custom_sw)
  nltk_tokenizer = nltk.WordPunctTokenizer().tokenize
  tokens = nltk_tokenizer(text)
  result = " ".join([i for i in tokens if not i in stop_words if len(i)>=1])
  return result

# Cleans a string by applying various pre-processing steps
def clean_text(text):
  new_text = make_lowercase(text)
  new_text = expand_contractions(new_text)
  new_text = remove_punct(new_text)
  return new_text

def correct_spelling(text):
  new_text = str(TextBlob(text).correct())
  return new_text

def make_lowercase(text):
  new_text = text.lower()
  return new_text

def expand_contractions(text):
  new_text = contractions.fix(text)
  return new_text

def remove_punct(text):
  remove_punct_map = dict.fromkeys(map(ord, string.punctuation))
  text.translate(remove_punct_map)
  return text

def remove_numbers(text):
  new_text =re.sub("\d+", "", text)
  return new_text

clear_output()
df = pd.DataFrame.from_csv('rship.csv')
df.dropna(subset=['title'], inplace=True)
documents = df['title']
clean_docs = [clean_text(i) for i in documents]
clean_docs = [tokenize(i) for i in documents]
 